## Importation des données

In [5]:
import sys
from pathlib import Path
import importlib
import ipynbname 
import pandas as pd
import geopandas as gpd
from datetime import datetime

code_path = ipynbname.path().parent.parent
# Ajouter le dossier scripts au path
scripts_path = code_path  / "scripts"
base_path=code_path.parent
sys.path.append(str(scripts_path.resolve()))

import data_utils  # importe le module une première fois

# Après avoir modifié data_utils.py
importlib.reload(data_utils)

# Maintenant tu peux accéder aux fonctions mises à jour
from data_utils import import_data_raw, import_data_sig, melt_long_format, clean_year_column, save_long_dataframe, concat_intermediate_files


In [6]:
#Country data
filename="data_final_all_norm.csv"
filepath= base_path/ "Data" / 'data_final' / filename

df_data = pd.read_csv(filepath)
df_data.head(100)

,Year,Country,Value,Unit,Indicator,Source,Country_code,country,gdp_percapita_current_usd,gdp_percapita_ppp_current_intl,...,Value_norm_gdp,Unit_norm_gdp,Value_norm_ppp,Unit_norm_ppp,Value_norm_gdp_hab,Unit_norm_gdp_hab,Value_norm_ppp_hab,Unit_norm_ppp_hab,Value_norm_densite,Unit_norm_densite
0,1960,Afghanistan,4.17891,TgC/year,LULUCF Net emissions,GCB,AF,Afghanistan,NaN,NaN,...,NaN,TgC/year/billion USD,NaN,TgC/year/billion PPP$,NaN,TgC/year/thousand USD/hab,NaN,TgC/year/thousand PPP$/hab,NaN,TgC/year/hab/km²
1,1961,Afghanistan,3.59662,TgC/year,LULUCF Net emissions,GCB,AF,Afghanistan,NaN,NaN,...,NaN,TgC/year/billion USD,NaN,TgC/year/billion PPP$,NaN,TgC/year/thousand USD/hab,NaN,TgC/year/thousand PPP$/hab,0.254591,TgC/year/hab/km²
2,1962,Afghanistan,3.32416,TgC/year,LULUCF Net emissions,GCB,AF,Afghanistan,NaN,NaN,...,NaN,TgC/year/billion USD,NaN,TgC/year/billion PPP$,NaN,TgC/year/thousand USD/hab,NaN,TgC/year/thousand PPP$/hab,0.230543,TgC/year/hab/km²
3,1963,Afghanistan,3.23023,TgC/year,LULUCF Net emissions,GCB,AF,Afghanistan,NaN,NaN,...,NaN,TgC/year/billion USD,NaN,TgC/year/billion PPP$,NaN,TgC/year/thousand USD/hab,NaN,TgC/year/thousand PPP$/hab,0.219361,TgC/year/hab/km²
4,1964,Afghanistan,2.79609,TgC/year,LULUCF Net emissions,GCB,AF,Afghanistan,NaN,NaN,...,NaN,TgC/year/billion USD,NaN,TgC/year/billion PPP$,NaN,TgC/year/thousand USD/hab,NaN,TgC/year/thousand PPP$/hab,0.185820,TgC/year/hab/km²
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1991,Albania,0.07904,TgC/year,LULUCF Net emissions,GCB,AL,Albania,336.586995,1988.181406,...,0.071883,TgC/year/billion USD,0.012169,TgC/year/billion PPP$,0.234828,TgC/year/thousand USD/hab,0.039755,TgC/year/thousand PPP$/hab,0.000663,TgC/year/hab/km²
96,1992,Albania,0.04298,TgC/year,LULUCF Net emissions,GCB,AL,Albania,200.852220,1898.820304,...,0.065903,TgC/year/billion USD,0.006971,TgC/year/billion PPP$,0.213988,TgC/year/thousand USD/hab,0.022635,TgC/year/thousand PPP$/hab,0.000363,TgC/year/hab/km²
97,1993,Albania,-0.06407,TgC/year,LULUCF Net emissions,GCB,AL,Albania,367.279225,2142.681185,...,-0.054053,TgC/year/billion USD,-0.009265,TgC/year/billion PPP$,-0.174445,TgC/year/thousand USD/hab,-0.029902,TgC/year/thousand PPP$/hab,-0.000544,TgC/year/hab/km²
98,1994,Albania,-0.17971,TgC/year,LULUCF Net emissions,GCB,AL,Albania,586.416135,2384.728785,...,-0.095542,TgC/year/billion USD,-0.023494,TgC/year/billion PPP$,-0.306455,TgC/year/thousand USD/hab,-0.075359,TgC/year/thousand PPP$/hab,-0.001535,TgC/year/hab/km²


In [7]:
#Global sig
gdf_world=import_data_sig('world.geojson',base_path)


def simplify_geom(geom, tol=0.1):
    if geom is None:
        return None
    if geom.geom_type == 'Polygon':
        return geom.simplify(tol, preserve_topology=True)
    elif geom.geom_type == 'MultiPolygon':
        return type(geom)([poly.simplify(tol, preserve_topology=True) for poly in geom.geoms])
    return geom
    
gdf_world = gdf_world[gdf_world['Country_code'].notna()].copy()
gdf_world['geometry'] = gdf_world['geometry'].apply(lambda g: simplify_geom(g, tol=0.1))
gdf_world.head(10)


,geo_point_2d,iso3,status,color_code,name,continent,region,Country_code,french_short,Country,geometry
2,"{ ""lon"": 9.5613358449883421, ""lat"": 34.1108585...",TUN,Member State,TUN,Tunisia,Africa,Northern Africa,TN,Tunisie,Tunisia,"MULTIPOLYGON (((10.96833 33.8575, 10.865 33.63..."
3,"{ ""lon"": 43.77213543247138, ""lat"": 33.04802449...",IRQ,Member State,IRQ,Iraq,Asia,Western Asia,IQ,Iraq,Iraq,"POLYGON ((44.78734 37.14971, 44.85304 36.79458..."
4,"{ ""lon"": -6.3178452255610269, ""lat"": 31.883624...",MAR,Member State,MAR,Morocco,Africa,Northern Africa,MA,Maroc,Morocco,"POLYGON ((-2.94694 35.32916, -2.87694 35.1418,..."
5,"{ ""lon"": 25.088778010977098, ""lat"": -28.993203...",ZAF,Member State,ZAF,South Africa,Africa,Southern Africa,ZA,Afrique du Sud,South Africa,"POLYGON ((31.2975 -22.41476, 31.55083 -23.4766..."
6,"{ ""lon"": -61.253176819842295, ""lat"": 10.468641...",TTO,Member State,TTO,Trinidad and Tobago,Americas,Caribbean,TT,Trinité-et-Tobago,Trinidad and Tobago,"MULTIPOLYGON (((-60.92306 10.79722, -61.00445 ..."
7,"{ ""lon"": 2.5519552167777979, ""lat"": 46.5645020...",FRA,Member State,FRA,France,Europe,Western Europe,FR,France,France,"MULTIPOLYGON (((9.45958 42.98805, 9.55257 42.1..."
9,"{ ""lon"": 168.62692074653282, ""lat"": 7.64327257...",MHL,Member State,MHL,Marshall Islands,Oceania,Micronesia,MH,Îles Marshall,Marshall Islands,"MULTIPOLYGON (((168.13098 5.62236, 168.09497 5..."
10,"{ ""lon"": 12.071742970356828, ""lat"": 42.7957823...",ITA,Member State,ITA,Italy,Europe,Southern Europe,IT,Italie,Italy,"MULTIPOLYGON (((15.64794 38.26457, 15.0925 37...."
11,"{ ""lon"": -53.089819954013173, ""lat"": -10.77310...",BRA,Member State,BRA,Brazil,Americas,South America,BR,Brésil,Brazil,"MULTIPOLYGON (((-48.55056 -27.82139, -48.51778..."
12,"{ ""lon"": 29.871837767180068, ""lat"": -19.000098...",ZWE,Member State,ZWE,Zimbabwe,Africa,Eastern Africa,ZW,Zimbabwe,Zimbabwe,"POLYGON ((30.41576 -15.63187, 30.42236 -16.005..."


## Visualisation des données

In [20]:
import plotly.graph_objects as go
import pandas as pd
import geopandas as gpd
import json
import webbrowser
from pathlib import Path

def plot_world_map_with_slider(df_data, gdf_world, indicator, output_html="map_slider", save=True,base_path='C:/Users/Aubin/Documents/NetZero'):
    """
    Carte du monde interactive avec un slider pour choisir l'année manuellement.
    Colorbar : vert <0, blanc =0, rouge >0.
    Si save=True, le fichier est sauvegardé dans NetZero/Résultats.
    """
    date_str = datetime.now().strftime("%Y%m%d")
    output_html=f"{output_html}_{date_str}.html"
    # Définir le dossier de sauvegarde : NetZero/Figures
    output_path = base_path/'Figures' / output_html

    # Filtrer pour l'indicateur choisi
    df_filtered = df_data[df_data['Value'] == indicator]
    
    # Fusionner avec le GeoDataFrame
    gdf_merged = gdf_world.merge(df_filtered, on='Country_code', how='left')
    
    # Conversion des géométries en GeoJSON
    #gdf_merged = gdf_merged.set_geometry('geometry')
    geojson_data = json.loads(gdf_merged.to_json())
    
    # Déterminer les bornes min et max pour la colorbar
    zmin = df_filtered['Value'].min()
    zmax = df_filtered['Value'].max()
    zmed = abs(zmin / (zmax - zmin)) if zmax != zmin else 0.5  # éviter division par zéro
    
    # Liste des années disponibles
    years = sorted(df_filtered['Year'].unique())
    
    # Création des traces, une par année
    traces = []
    for i, year in enumerate(years):
        df_year = gdf_merged[gdf_merged['Year'] == year]
        traces.append(
            go.Choropleth(
                geojson=geojson_data,
                locations=df_year.index,
                z=df_year['Value'],
                text=df_year['name'],
                colorscale=[[0, "green"], [zmed, "white"], [1, "red"]],
                zmin=zmin,
                zmax=zmax,
                zmid=0,  # blanc = 0
                colorbar_title=indicator,
                visible=(i==0)  # seule la première année visible au départ
            )
        )

    # Création des étapes du slider
    steps = []
    for i, year in enumerate(years):
        step = dict(
            method="update",
            args=[{"visible": [j==i for j in range(len(traces))]}],
            label=str(year)
        )
        steps.append(step)
    
    # Slider
    sliders = [dict(
        active=0,
        currentvalue={"prefix": "Année: ", "font": {"size": 12}},
        pad={"t": 20},
        steps=steps
    )]

    fig = go.Figure(data=traces)
    fig.update_layout(
        title_text=f"{indicator} par année",
        geo=dict(
            scope="world",
            projection_type="natural earth",
            showcountries=True,
            showcoastlines=True,
            showland=True,
            landcolor="lightgray",
            fitbounds="locations"
        ),
        sliders=sliders,
        margin={"r":0, "t":50, "l":0, "b":0}
    )
    
    # Sauvegarder et/ou ouvrir automatiquement
    if save:
        # Récupérer la date au format YYYYMMDD
        # Nom du fichier avec la date
        fig.write_html(output_path)
        fig.write_html(output_path, include_plotlyjs='cdn')
        print(f"Carte interactive sauvegardée dans {output_path}")
    
    webbrowser.open_new_tab(output_path)


In [21]:
import webbrowser

# Nom du fichier HTML de sortie
output_file = "map_LULUCF_slider_light"

# Appel de la fonction avec slider pour l'année
plot_world_map_with_slider(
    df_data[df_data['Year'].isin([1980,1990,2000,2010,2020])],
    gdf_world=gdf_world,
    indicator="LULUCF Net emissions",
    output_html=output_file,
    save=True,
    base_path=base_path
)


Carte interactive sauvegardée dans C:\Users\Aubin\Documents\NetZero\Figures\map_LULUCF_slider_light_20251111.html


In [17]:
df_data

,countryiso3,Year,country,gdp_percapita_current_usd,gdp_percapita_ppp_current_intl,missing_population_total,missing_gdp_current_usd,missing_gdp_percapita_current_usd,missing_gdp_ppp_current_intl,missing_gdp_percapita_ppp_current_intl,...,Value_norm_gdp,Unit_norm_gdp,Value_norm_ppp,Unit_norm_ppp,Value_norm_gdp_hab,Unit_norm_gdp_hab,Value_norm_ppp_hab,Unit_norm_ppp_hab,Value_norm_densite,Unit_norm_densite
0,1A,1960,Arab World,NaN,NaN,False,True,True,True,True,...,NaN,/billion USD,NaN,/billion PPP$,NaN,/thousand USD/hab,NaN,/thousand PPP$/hab,NaN,/hab/km²
1,1A,1961,Arab World,212.889663,NaN,False,False,False,True,True,...,19.997084,/billion USD,NaN,/billion PPP$,0.212890,/thousand USD/hab,NaN,/thousand PPP$/hab,8.360295,/hab/km²
2,1A,1962,Arab World,210.805415,NaN,False,False,False,True,True,...,20.327671,/billion USD,NaN,/billion PPP$,0.210805,/thousand USD/hab,NaN,/thousand PPP$/hab,8.582530,/hab/km²
3,1A,1963,Arab World,225.800630,NaN,False,False,False,True,True,...,22.362958,/billion USD,NaN,/billion PPP$,0.225801,/thousand USD/hab,NaN,/thousand PPP$/hab,8.814823,/hab/km²
4,1A,1964,Arab World,243.897432,NaN,False,False,False,True,True,...,24.811627,/billion USD,NaN,/billion PPP$,0.243897,/thousand USD/hab,NaN,/thousand PPP$/hab,9.054355,/hab/km²
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17285,ZW,2020,Zimbabwe,1730.453910,3510.676040,False,False,False,False,False,...,26.868564,/billion USD,54.509874,/billion PPP$,1.730454,/thousand USD/hab,3.510676,/thousand PPP$/hab,40.136714,/hab/km²
17286,ZW,2021,Zimbabwe,1724.387271,3184.784602,False,False,False,False,False,...,27.240508,/billion USD,50.310711,/billion PPP$,1.724387,/thousand USD/hab,3.184785,/thousand PPP$/hab,40.835492,/hab/km²
17287,ZW,2022,Zimbabwe,2040.546587,3560.039403,False,False,False,False,False,...,32.789657,/billion USD,57.206473,/billion PPP$,2.040547,/thousand USD/hab,3.560039,/thousand PPP$/hab,41.538209,/hab/km²
17288,ZW,2023,Zimbabwe,2156.034093,3820.359922,False,False,False,False,False,...,35.231369,/billion USD,62.427821,/billion PPP$,2.156034,/thousand USD/hab,3.820360,/thousand PPP$/hab,42.240719,/hab/km²


## with button / normalisation

In [26]:
import plotly.graph_objects as go
import pandas as pd
import geopandas as gpd
import json
import webbrowser
from pathlib import Path
from datetime import datetime
import numpy as np

def symlog(x):
    """Logarithme symétrique : log10(|x|+1) * signe(x)"""
    if pd.isna(x):
        return None
    return np.sign(x) * np.log10(abs(x) + 1)

def plot_world_map_button(df_data, gdf_world, indicator, year,
                          output_html="map_buttons",
                          save=True,
                          base_path=Path('C:/Users/Aubin/Documents/NetZero'),
                          scale="relative",
                          color_range="raw",
                          color_map="RdYlGn_r"):

    # --- Préparer sortie ---
    date_str = datetime.now().strftime("%Y%m%d")
    output_html = f"{output_html}_{scale}_{color_range}_{date_str}.html"
    output_path = base_path / 'Figures' / output_html
    output_path.parent.mkdir(parents=True, exist_ok=True)

    # --- Nettoyer gdf_world ---
    gdf_world = gdf_world[gdf_world["Country_code"].notna()].copy()

    # --- Filtrer indicateur et année ---
    df_filtered = df_data[(df_data['Indicator'] == indicator) & (df_data['Year'] == year)].copy()
    # --- Fusion avec GeoDataFrame ---
    gdf_merged = gdf_world.merge(df_filtered, on='Country_code', how='left')
    geojson_data = json.loads(gdf_merged.to_json())

    # --- Normalisations et colonnes d'unité ---
    norm_map = {
        'No norm': ('Value', 'Unit'),
        'Area': ('Value_norm_area', 'Unit_norm_area'),         # area / million km²
        'Population': ('Value_norm_population', 'Unit_norm_population'),  # population / million inhabitants
        'Hab/km2': ('Value_norm_densite', 'Unit_norm_densite'),   # population density / hab/km²
        'PPP': ('Value_norm_ppp', 'Unit_norm_ppp'),           # GDP PPP total / billion PPP$
        'GDP': ('Value_norm_gdp', 'Unit_norm_gdp'),           # GDP total / billion USD
        'PPP/hab': ('Value_norm_ppp_hab', 'Unit_norm_ppp_hab'),  # GDP PPP per capita / thousand PPP$/hab
        'GDP/hab': ('Value_norm_gdp_hab', 'Unit_norm_gdp_hab')  # GDP per capita / thousand USD/hab
    }

    # Colonne pour le hover : la valeur brute issue de df_filtered
    #hover_values = gdf_merged[col_value]  # col_value correspond à la colonne "Value" ou normalisée

    # --- Fonction pour bornes de couleur ---
    def compute_zrange(col_values):
        if color_range == 'raw':
            return col_values.min(), col_values.max(), col_values
        elif color_range.startswith("q"):
            q_low = float(color_range[1:5])
            q_high = 1 - q_low
            return col_values.quantile(q_low), col_values.quantile(q_high), col_values
        elif color_range.startswith("*"):
            factor = float(color_range[1:])
            return factor * col_values.min(), factor * col_values.max(), col_values
        else:
            raise ValueError("color_range must be 'raw', 'q0.xx', '*0.xx'")

    # --- Créer toutes les traces (une par normalisation) ---
    traces = []
    for norm_name, (col_value, col_unit) in norm_map.items():
        zmin, zmax, z_values = compute_zrange(df_filtered[col_value])
        unit_val = df_filtered[col_unit].iloc[0] if col_unit in df_filtered.columns else ""
        colorbar_title = f"{indicator} ({unit_val})"

        z_plot = gdf_merged[col_value]

        if scale == 'rank':
            ranks = z_plot.copy()
            mask = ranks.notna()
            ranks[mask] = ranks[mask].rank(ascending=True)
            
            trace = go.Choropleth(
                geojson=geojson_data,
                locations=gdf_merged.index,
                z=ranks,
                text=gdf_merged['name'],
                colorscale=color_map,
                zmin=ranks.min(),
                zmax=ranks.max(),
                customdata=gdf_merged[col_value],
                colorbar_title=f"{colorbar_title} (Rank)",
                hovertemplate = (f"<b>%{{text}}</b><br>{indicator} ({norm_name}) = %{{customdata:.2f}} {unit_val}<extra></extra>"),
                visible=(norm_name == "No norm"),
                name=norm_name
            )

        elif scale == 'absolute':
            zmid = abs(zmin / (zmax - zmin)) if zmax != zmin else 0.5
            trace = go.Choropleth(
                geojson=geojson_data,
                locations=gdf_merged.index,
                z=z_plot,
                text=gdf_merged['name'],
                colorscale=[[0, "green"], [zmid, "yellow"], [1, "red"]],
                zmin=-zmax,
                zmax=-zmax,
                zmid=0,
                customdata=gdf_merged[col_value],
                colorbar_title=colorbar_title,
                hovertemplate = (f"<b>%{{text}}</b><br>{indicator} ({norm_name}) = %{{customdata:.2f}} {unit_val}<extra></extra>"),
                visible=(norm_name == "No norm"),
                name=norm_name
            )

        elif scale == 'relative':
            trace = go.Choropleth(
                geojson=geojson_data,
                locations=gdf_merged.index,
                z=z_plot,
                text=gdf_merged['name'],
                colorscale=color_map,
                zmin=zmin,
                zmax=zmax,
                customdata=gdf_merged[col_value],
                colorbar_title=colorbar_title,
                hovertemplate = (f"<b>%{{text}}</b><br>{indicator} ({norm_name}) = %{{customdata:.2f}} {unit_val}<extra></extra>"),
                visible=(norm_name == "No norm"),
                name=norm_name
            )

        elif scale == 'log':
            z_log = z_plot.apply(symlog)
            trace = go.Choropleth(
                geojson=geojson_data,
                locations=gdf_merged.index,
                z=z_log,
                customdata=gdf_merged[col_value],
                text=gdf_merged['name'],
                colorscale=color_map,
                colorbar_title=f"{colorbar_title} (symlog10)",
                hovertemplate = (f"<b>%{{text}}</b><br>{indicator} ({norm_name}) = %{{customdata:.2f}} {unit_val}<extra></extra>"),
                visible=(norm_name == "No norm"),
                name=norm_name
            )

        else:
            raise ValueError("scale must be 'absolute', 'relative','rank' or 'log'")

        traces.append(trace)

    # --- Boutons pour normalisation (titre reste constant) ---
    buttons = []
    for i, norm_name in enumerate(norm_map.keys()):
        visible_list = [False] * len(traces)
        visible_list[i] = True
        buttons.append(dict(
            label=norm_name,
            method="update",
            args=[{"visible": visible_list}]
        ))

    updatemenus = [dict(
        active=0,
        buttons=buttons,
        x=0.0,
        y=1.05,
        xanchor='left',
        yanchor='top'
    )]

    # --- Création figure ---
    subtitle = f"Scale: {scale}, Color range: {color_range}"
    fig = go.Figure(data=traces)
    fig.update_layout(
        title=dict(
            text=f"{indicator} - Année {year}<br><sup>{subtitle}</sup>",
            x=0.5,
            xanchor='center'
        ),
        geo=dict(
            scope="world",
            projection_type="natural earth",
            showcountries=True,
            showcoastlines=True,
            showland=True,
            showocean=True,          # ✅ show oceans
            landcolor="lightgray",
            oceancolor="lightblue",  # ✅ ocean color
            fitbounds="locations"
        ),
        updatemenus=updatemenus,
        margin={"r": 0, "t": 80, "l": 0, "b": 0}
    )

    # --- Sauvegarde et ouverture ---
    if save:
        fig.write_html(output_path, include_plotlyjs='cdn')
        print(f"Carte interactive sauvegardée dans {output_path}")
        webbrowser.open_new_tab(output_path)


In [27]:
import webbrowser

# Nom du fichier HTML de sortie
output_file = "map_LULUCF_button_test"

# Appel de la fonction avec slider pour l'année
plot_world_map_button(
    df_data,
    gdf_world=gdf_world,
    indicator="LULUCF Net emissions",
    year=2020,
    output_html=output_file,
    save=True,
    base_path=base_path,
    scale="rank", #absolute, relative, log, rank
    color_range='raw',
    color_map="YlOrBr"#raw, q0.01, *0.9
)

Carte interactive sauvegardée dans C:\Users\Aubin\Documents\NetZero\Figures\map_LULUCF_button_test_rank_raw_20251112.html


## with dash

In [125]:
norm_map = {
    'No norm': 'Value',
    'PPP': 'Value_norm_ppp',
    'GDP': 'Value_norm_gdp',
    'Hab': 'Value_norm_person',
    'Area': 'Value_norm_area'
}
n_norm = len(norm_map)
for i_norm, norm_name in enumerate(norm_map.keys()):
    print (i_norm)

0
1
2
3
4


In [134]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import pandas as pd
import geopandas as gpd
import json
import numpy as np

def symlog(x):
    if pd.isna(x):
        return None
    return np.sign(x) * np.log10(abs(x)+1)

# --- Example Data ---
# df_data: columns = ['Country_code', 'Year', 'Indicator', 'Value', 'Value_norm_ppp', ...]
# gdf_world: geopandas dataframe with geometry and 'Country_code'

# Replace these with your actual dataframes
df_data = pd.read_csv("your_data.csv")
gdf_world = gpd.read_file("your_shapefile.shp")

# --- Normalizations ---
norm_map = {
    'No norm': 'Value',
    'PPP': 'Value_norm_ppp',
    'GDP': 'Value_norm_gdp',
    'Hab': 'Value_norm_person',
    'Area': 'Value_norm_area'
}

years = sorted(df_data['Year'].unique())
norm_names = list(norm_map.keys())

# --- Prepare merged GeoDataFrame ---
def get_gdf_for_year_norm(year, norm_name):
    col = norm_map[norm_name]
    df_filtered = df_data[df_data['Year'] == year]
    gdf_merged = gdf_world.merge(df_filtered, on='Country_code', how='left')
    return gdf_merged, col

# --- Dash App ---
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("World Map with Slider and Normalization"),
    
    html.Div([
        html.Label("Select Normalization:"),
        dcc.Dropdown(
            id='norm-dropdown',
            options=[{'label': n, 'value': n} for n in norm_names],
            value='No norm'
        )
    ], style={'width': '30%', 'display': 'inline-block'}),
    
    html.Div([
        html.Label("Select Year:"),
        dcc.Slider(
            id='year-slider',
            min=min(years),
            max=max(years),
            value=min(years),
            marks={y: str(y) for y in years},
            step=None
        )
    ], style={'width': '60%', 'display': 'inline-block', 'padding': '0px 20px 20px 20px'}),
    
    dcc.Graph(id='choropleth-map')
])

@app.callback(
    Output('choropleth-map', 'figure'),
    Input('year-slider', 'value'),
    Input('norm-dropdown', 'value')
)
def update_map(selected_year, selected_norm):
    gdf_merged, col = get_gdf_for_year_norm(selected_year, selected_norm)
    geojson_data = json.loads(gdf_merged.to_json())
    
    zmin, zmax = gdf_merged[col].min(), gdf_merged[col].max()
    
    fig = go.Figure(go.Choropleth(
        geojson=geojson_data,
        locations=gdf_merged.index,
        z=gdf_merged[col],
        text=gdf_merged['name'] if 'name' in gdf_merged.columns else gdf_merged['Country_code'],
        colorscale="RdYlGn_r",
        zmin=zmin,
        zmax=zmax,
        colorbar_title=selected_norm
    ))
    
    fig.update_layout(
        title=f"{selected_norm} - Year {selected_year}",
        geo=dict(
            scope="world",
            projection_type="natural earth",
            showcountries=True,
            showcoastlines=True,
            showland=True,
            landcolor="lightgray"
        ),
        margin={"r":0, "t":50, "l":0, "b":0}
    )
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)


FileNotFoundError: [Errno 2] No such file or directory: 'your_data.csv'

In [135]:
import dash
from dash import dcc, html
import pandas as pd
import geopandas as gpd

# Filter your data for the years you want
df_filtered = df_data[df_data['Year'].isin([2000, 2020])]

# Optionally, define the output file if you still want to save HTML later
output_file = "map_LULUCF_slider_test.html"

# Run the Dash app
# The Dash app will automatically use df_filtered as the data source
# You can modify the get_gdf_for_year_norm function to use df_filtered instead of df_data

app = dash.Dash(__name__)

# Reuse the previous Dash layout and callback, but with df_filtered instead of df_data
# Make sure to pass df_filtered to the callback / helper functions

if __name__ == '__main__':
    app.run_server(debug=True)


ObsoleteAttributeException: app.run_server has been replaced by app.run